In [1]:
#以下のような回転をさせるのかと思いましたが、ずらすとaverageImageがぼやけすぎます。

import keras
from keras.datasets import mnist
import numpy as np
from PIL import Image

(x_train, y_train), (x_test, y_test) = mnist.load_data()

img = Image.fromarray(np.uint8(x_test[44]))
originalImgSizeW = 100
originalImgSizeH = 100
img_resize = img.resize((originalImgSizeW, originalImgSizeH))
img_resize.save("Original.jpg")

imgVectors = []

imgSizeW = 100
imgSizeH = 100
samplingNum = 10000
mu = np.array([0.0, 0.0, 0.0])
sigma = np.diag([10.0**2, 10.0**2, 5.0**2])
positions = np.random.multivariate_normal(mu, sigma, samplingNum)

for p in positions:
    img_rotate = img_resize.rotate(p[2])
    (left, upper, right, lower) = (int(p[0]) + originalImgSizeW / 2 - imgSizeW / 2, 
                                   int(p[1]) + originalImgSizeH / 2 - imgSizeH / 2,
                                   int(p[0]) + originalImgSizeW / 2 + imgSizeW / 2, 
                                   int(p[1]) + originalImgSizeH / 2 + imgSizeH / 2)
    img_crop = img_rotate.crop((left, upper, right, lower))
    
    #If samplingNum is big, this code should not be activated.
    #img_crop.show()
    
    imgVectors.append(np.array(np.resize(np.array(img_crop), (1, imgSizeW * imgSizeH))[0]))
    
averageData = sum(np.uint32(np.array(imgVectors))) / samplingNum
print(np.resize(averageData, (imgSizeW, imgSizeH)))
print(max(averageData))
print(min(averageData))
averageImg = Image.fromarray(np.uint8(np.resize(averageData, (imgSizeW, imgSizeH))))
averageImg.save("average.jpg")

[[0.     0.     0.     ... 0.     0.     0.    ]
 [0.     0.     0.0004 ... 0.     0.     0.    ]
 [0.0004 0.0013 0.0028 ... 0.     0.     0.    ]
 ...
 [0.0226 0.028  0.0412 ... 0.     0.     0.    ]
 [0.0092 0.0114 0.023  ... 0.     0.     0.    ]
 [0.     0.     0.0073 ... 0.     0.     0.    ]]
116.162
0.0


In [2]:
#様々なテスト
#kerasを初めて使った。
#PILは使ったことはあったが、使い方はよくわかっていない。
#OpenCVのほうが良かった？

import keras
from keras.datasets import mnist
import numpy as np
from PIL import Image

# 文字画像表示
def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

# Kerasの関数でデータの読み込み。データをシャッフルして学習データと訓練データに分割
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# MNISTの文字画像を読み出して表示
num = 100
print(len(x_train))
print(len(x_test))

#図12.1と図12.5はテストデータの44番目が近そう。
#img_show(x_test[44].reshape(28, 28))
#print(x_test[44])
#print(x_test[44].reshape(1, 28*28))

img = Image.fromarray(np.uint8(x_test[44]))
img_resize = img.resize((64, 64))
img_rotate = img_resize.rotate(45)

#print(img_resize)

#img_resize.show()
#img_rotate.show()

for i in range(num):
    #テストデータの4番目が近そう
#    if y_test[i] == 3:
#        img_show(x_test[i].reshape(28, 28))
#        print(i)

    if y_train[i] == 3:
        img_show(x_train[i].reshape(28, 28))
        print(i)

(left, upper, right, lower) = (20, 20, 64, 64)

img_crop = img_rotate.crop((left, upper, right, lower))
#img_crop.show()

raw_img = np.array(img_crop)
#print(raw_img)
#print(raw_img.shape)

60000
10000
7
10
12
27
30
44
49
50
74
86
98


In [3]:
import numpy as np
mu = np.array([0.0, 0.0, 0.0])
sigma = np.diag([10.0**2, 10.0**2, 10.0**2])
print(sigma)

values = np.random.multivariate_normal(mu, sigma, 10)
print(values)
print(np.round(values))    

[[100.   0.   0.]
 [  0. 100.   0.]
 [  0.   0. 100.]]
[[  0.32639593   4.03081699  14.84189485]
 [-16.05834512   2.26741858  -0.72586689]
 [-14.72499121 -18.59752836  16.71245402]
 [ -0.59212272   8.00640407  -5.16436458]
 [  1.5234884   -2.026979    27.5355176 ]
 [-10.58177163  11.1767858    0.97930713]
 [  3.81131414   8.75358912  -2.51901599]
 [ 16.96478053   0.20733184  -6.12147521]
 [ 18.91571032   3.78577625   8.42156852]
 [ -2.82570038   5.0202498   -9.12721081]]
[[  0.   4.  15.]
 [-16.   2.  -1.]
 [-15. -19.  17.]
 [ -1.   8.  -5.]
 [  2.  -2.  28.]
 [-11.  11.   1.]
 [  4.   9.  -3.]
 [ 17.   0.  -6.]
 [ 19.   4.   8.]
 [ -3.   5.  -9.]]


In [4]:
#IDOはどこへ行った？
import keras
from keras.datasets import mnist
import numpy as np
from PIL import Image

def displayVector(vec, width, height):
    img = Image.fromarray(np.uint8(np.resize(vec, (width, height))))
    img.show()

def normalizeVector(vec):
    return np.uint8((vec + 1.0) / 2.0 * 255)
    
(x_train, y_train), (x_test, y_test) = mnist.load_data()

imageSizeW = len(x_train[0][0])
imageSizeH = len(x_train[0])

imageData = []

for x, y in zip(x_train, y_train):
    if y == 3:
        imageData.append(np.array(Image.fromarray(np.uint8(x))))

averageData = sum(np.uint32(imageData)) / len(imageData)

averageImage = Image.fromarray(np.uint8(averageData))
averageImage.save("average.png")

imageVectors = [np.resize(image, (1, imageSizeW*imageSizeH))[0] for image in imageData]

#S = np.zeros((imageSizeW*imageSizeH, imageSizeW*imageSizeH))

xBar = np.array(np.resize(averageData, (1, imageSizeW*imageSizeH))[0])
displayVector(xBar, imageSizeW, imageSizeH)

X = np.array(imageVectors)
S = (X - xBar).T.dot(X - xBar) / len(imageVectors)

eigs, eigVectors = np.linalg.eig(S)
#Sは正定値のハズだが計算誤差の影響か、負が混じっている。
#print(eigs)
print(eigVectors.T[0])
#固有値は転置が必要らしい。
displayVector(normalizeVector(eigVectors.T[0]), imageSizeW, imageSizeH)


[ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  if sys.path[0] == '':


In [5]:
A = np.array([[1, 2], [3, 4]])
a = np.array([1, 2])
print(A-a)

[[0 0]
 [2 2]]


In [6]:
displayVector(normalizeVector(eigVectors.T[3]), imageSizeW, imageSizeH)

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  if sys.path[0] == '':
